<a href="https://colab.research.google.com/github/hsandaver/essays/blob/main/DyeClassifierv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Conda and RDKit in Colab
# Note: This step is optional if you already have a proper RDKit installation.
# Uncomment the following lines if you need to install RDKit via Conda.

# !wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x Miniconda3-latest-Linux-x86_64.sh
# !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# !conda update -q conda
# import sys
# sys.path.append('/usr/local/lib/python3.10/site-packages/')
# !conda install -y -c rdkit rdkit

# Install RDKit via pip if not using Conda
!pip install rdkit-pypi pandas

# Import necessary libraries
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from google.colab import files
import re
import warnings

# Suppress RDKit warnings for cleaner output
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings("ignore")

# Define SMARTS patterns for chromophore detection
chromophore_patterns = {
    'Azo': Chem.MolFromSmarts('N=N'),  # Azo group
    'Anthraquinone': Chem.MolFromSmarts('C1=CC=C2C(=O)C=CC2=O'),  # Anthraquinone
    'Nitro': Chem.MolFromSmarts('[NX3](=O)=O'),  # Nitro group
    'Quinone': Chem.MolFromSmarts('O=C1C=CC=CC1=O'),  # Quinone
    'Indigoid': Chem.MolFromSmarts('C1C=CC(=O)NC1=O'),  # Indigoid pattern
    'Cyanine': Chem.MolFromSmarts('C=C-C=C'),  # Cyanine (simplified)
    'Xanthene': Chem.MolFromSmarts('O1C=CC2=C1C=CC=C2'),  # Xanthene (simplified)
    'Thiazine': Chem.MolFromSmarts('N1C=NC=S1'),  # Thiazine (sulfur-based)
    'Coumarin': Chem.MolFromSmarts('O=C1OC=CC2=CC=CC=C12'),  # Coumarin
    'Porphyrin': Chem.MolFromSmarts('N1C=CC=N1'),  # Porphyrin ring
    'Phthalocyanine': Chem.MolFromSmarts('C1=C(C2=NC=C(C)N2)C3=CC=CC=C13'),  # Phthalocyanine-like
    'Carotenoid': Chem.MolFromSmarts('C=C(C)C=CC=C'),  # Carotenoid (simplified)
    'Squaraine': Chem.MolFromSmarts('C=CC=C'),  # Squaraines (simplified conjugated)
    'Metal Complex': Chem.MolFromSmarts('[!#1]')  # Placeholder for metal coordination complexes
}

# New SMARTS patterns for heavy atoms and unusual ligands
additional_patterns = {
    'Bromine': Chem.MolFromSmarts('Br'),  # Bromine
    'Selenium': Chem.MolFromSmarts('Se'),  # Selenium
    'Pyridine': Chem.MolFromSmarts('C1=CC=NC=C1'),  # Pyridine ligand
    'Phosphine': Chem.MolFromSmarts('P(C)(C)C'),  # Phosphine ligand
    'Carbene': Chem.MolFromSmarts('[C]')  # Carbene ligand
}

# Combine all patterns into a single dictionary
chromophore_patterns.update(additional_patterns)

# Check if all SMARTS patterns are compiled correctly
for name, pattern in chromophore_patterns.items():
    if pattern is None:
        print(f"Warning: Failed to compile SMARTS pattern for {name}")

# Define SMARTS patterns for auxochromes
auxochrome_patterns = {
    'Hydroxyl': Chem.MolFromSmarts('[OX2H]'),  # -OH
    'Amine': Chem.MolFromSmarts('N'),  # -NH2 or other amines
    'Methoxy': Chem.MolFromSmarts('COC'),  # -OCH3
    'Thiol': Chem.MolFromSmarts('[SX2H]'),  # -SH
    'Carboxyl': Chem.MolFromSmarts('C(=O)[OX2H1]')
}

# Helper functions to calculate number of double bonds and rings
def calc_num_double_bonds(mol):
    """Calculate the number of double bonds in a molecule."""
    return sum(1 for bond in mol.GetBonds() if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE)

def calc_num_rings(mol):
    """Calculate the number of rings in a molecule."""
    return mol.GetRingInfo().NumRings()

# Function to attempt to correct SMILES strings
def correct_smiles(smiles):
    original_smiles = smiles
    corrected = False
    corrected_smiles = smiles

    # Attempt 1: Remove stereochemistry markers (/ and \)
    if '/' in corrected_smiles or '\\' in corrected_smiles:
        corrected_smiles = re.sub(r'[\\/]', '', corrected_smiles)
        corrected = True

    # Attempt 2: Balance ring numbers if possible
    # This is a simplistic approach; for complex corrections, more advanced parsing is needed
    ring_numbers = re.findall(r'\d', corrected_smiles)
    if len(ring_numbers) % 2 != 0:
        # Attempt to remove the last ring number
        corrected_smiles = re.sub(r'\d', '', corrected_smiles, count=1)
        corrected = True

    # Attempt 3: Additional corrections can be implemented here
    # For example, handling common errors like missing brackets or invalid charges

    return corrected_smiles, corrected

# Function to identify chromophores
def identify_chromophores(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return 'Invalid SMILES'

    matched_chromophores = []

    for name, pattern in chromophore_patterns.items():
        if pattern and mol.HasSubstructMatch(pattern):
            matched_chromophores.append(name)

    return ', '.join(matched_chromophores) if matched_chromophores else 'Unknown'

# Function to identify auxochromes
def identify_auxochromes(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return 'Invalid SMILES'

    matched_auxochromes = []

    for name, pattern in auxochrome_patterns.items():
        if pattern and mol.HasSubstructMatch(pattern):
            matched_auxochromes.append(name)

    return ', '.join(matched_auxochromes) if matched_auxochromes else 'None'

# Function to calculate descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        # Return None for all descriptors if SMILES is invalid
        return {
            'MolWeight': None,
            'LogP': None,
            'TPSA': None,
            'NumRings': None,
            'NumDoubleBonds': None
        }

    try:
        mol_weight = Descriptors.MolWt(mol)  # Molecular Weight
        logP = Descriptors.MolLogP(mol)      # Hydrophobicity (LogP)
        tpsa = rdMolDescriptors.CalcTPSA(mol)  # Topological Polar Surface Area (TPSA)
        num_rings = calc_num_rings(mol)        # Number of rings
        num_double_bonds = calc_num_double_bonds(mol)  # Number of double bonds
    except Exception as e:
        print(f"Error calculating descriptors for SMILES '{smiles}': {e}")
        return {
            'MolWeight': None,
            'LogP': None,
            'TPSA': None,
            'NumRings': None,
            'NumDoubleBonds': None
        }

    return {
        'MolWeight': mol_weight,
        'LogP': logP,
        'TPSA': tpsa,
        'NumRings': num_rings,
        'NumDoubleBonds': num_double_bonds
    }

# Function to map chromophores to color families
def chromophore_to_color(chromophore):
    chromophore_color_map = {
        'Azo': 'Red/Orange/Yellow',
        'Anthraquinone': 'Red/Blue/Violet',
        'Nitro': 'Yellow/Orange',
        'Quinone': 'Yellow/Orange/Brown',
        'Indigoid': 'Blue/Purple',
        'Cyanine': 'Green/Blue',
        'Xanthene': 'Yellow/Orange',
        'Thiazine': 'Blue/Green',
        'Coumarin': 'Blue/Green',
        'Porphyrin': 'Red/Purple',
        'Phthalocyanine': 'Green/Blue',
        'Carotenoid': 'Yellow/Orange',
        'Squaraine': 'Red/Purple',
        'Bromine': 'Dark Green/Purple',
        'Selenium': 'Deep Blue/Purple',
        'Pyridine': 'Varies (often Green/Blue/Yellow)',
        'Phosphine': 'Varies (Yellow/Green)',
        'Carbene': 'Varies (Red/Purple)',
        'Metal Complex': 'Varies (often Green/Blue)'
    }
    return chromophore_color_map.get(chromophore, 'Unknown')

# Function to estimate color based on chromophores, auxochromes, and descriptors
def estimate_color(chromophores, auxochromes, descriptors):
    if chromophores == 'Invalid SMILES':
        return 'Invalid SMILES'

    if chromophores == 'Unknown' and descriptors:
        # More nuanced inference rules
        try:
            if descriptors['NumDoubleBonds'] > 5:
                base_color = 'Red/Orange'
            elif descriptors['NumDoubleBonds'] > 3:
                base_color = 'Yellow/Orange'
            elif descriptors['NumRings'] >= 4:
                base_color = 'Blue/Violet'
            elif descriptors['NumRings'] == 3:
                base_color = 'Green/Blue'
            elif descriptors['MolWeight'] is not None and descriptors['MolWeight'] > 600:
                base_color = 'Deep Color (likely Red or Purple)'
            elif descriptors['MolWeight'] is not None and descriptors['MolWeight'] > 400:
                base_color = 'Moderate Color (likely Blue or Green)'
            else:
                base_color = 'Lighter Color (likely Yellow)'
        except TypeError:
            base_color = 'Unknown'
    else:
        # If multiple chromophores, take the first for base color
        first_chromophore = chromophores.split(', ')[0]
        base_color = chromophore_to_color(first_chromophore)

    # Modify the color based on auxochromes
    if auxochromes != 'Invalid SMILES':
        auxo_list = [auxo.strip() for auxo in auxochromes.split(',')]
        if 'Hydroxyl' in auxo_list:
            base_color += ' (Shifted towards Red)'
        if 'Amine' in auxo_list:
            base_color += ' (Shifted towards Blue/Violet)'
        if 'Methoxy' in auxo_list:
            base_color += ' (Shifted towards Yellow)'
        if 'Thiol' in auxo_list:
            base_color += ' (Potential for Increased Color Intensity)'
        if 'Carboxyl' in auxo_list:
            base_color += ' (Potential for Increased Solubility)'

    return base_color

# Function to visualize SMILES (optional)
def visualize_smiles(smiles):
    from rdkit.Chem import Draw
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Draw.MolToImage(mol)
    else:
        return None

# Allow user to upload file in Colab
print("Please upload a CSV file containing a 'SMILES' column.")
uploaded = files.upload()

# Check if any file was uploaded
if not uploaded:
    raise ValueError("No file uploaded. Please upload a CSV file with a 'SMILES' column.")

# Load the uploaded file into a pandas DataFrame
filename = next(iter(uploaded))
print(f"User uploaded file '{filename}'")
try:
    dye_data = pd.read_csv(filename)
except Exception as e:
    raise ValueError(f"Error reading the CSV file: {e}")

# Check if 'SMILES' column exists
if 'SMILES' not in dye_data.columns:
    raise ValueError("The uploaded CSV does not contain a 'SMILES' column.")

# Apply functions to the dataset
print("Processing data...")

# Initialize columns for correction status
dye_data['SMILES_Corrected'] = False
dye_data['SMILES_Valid'] = False
dye_data['Corrected_SMILES'] = dye_data['SMILES']

# Apply SMILES correction and validation
for idx, row in dye_data.iterrows():
    smiles = row['SMILES']
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        # Attempt to correct SMILES
        corrected_smiles, corrected = correct_smiles(smiles)
        dye_data.at[idx, 'Corrected_SMILES'] = corrected_smiles
        if corrected:
            dye_data.at[idx, 'SMILES_Corrected'] = True
            mol = Chem.MolFromSmiles(corrected_smiles)
            if mol:
                dye_data.at[idx, 'SMILES_Valid'] = True
            else:
                dye_data.at[idx, 'SMILES_Valid'] = False
        else:
            dye_data.at[idx, 'SMILES_Valid'] = False
    else:
        dye_data.at[idx, 'SMILES_Valid'] = True

# Identify chromophores
dye_data['Chromophore'] = dye_data['Corrected_SMILES'].apply(identify_chromophores)

# Identify auxochromes
dye_data['Auxochrome'] = dye_data['Corrected_SMILES'].apply(identify_auxochromes)

# Calculate descriptors
dye_data['Descriptors'] = dye_data['Corrected_SMILES'].apply(calculate_descriptors)

# Expand descriptors into separate columns
descriptor_df = dye_data['Descriptors'].apply(pd.Series)

# Define expected descriptor columns
expected_descriptors = ['MolWeight', 'LogP', 'TPSA', 'NumRings', 'NumDoubleBonds']

# Ensure all expected descriptor columns are present
for desc in expected_descriptors:
    if desc not in descriptor_df.columns:
        descriptor_df[desc] = None

dye_data = pd.concat([dye_data, descriptor_df], axis=1)
dye_data.drop(columns=['Descriptors'], inplace=True)

# Estimate color based on chromophores, auxochromes, and descriptors
dye_data['Estimated Color'] = dye_data.apply(
    lambda row: estimate_color(
        row['Chromophore'],
        row['Auxochrome'],
        {
            'MolWeight': row['MolWeight'],
            'LogP': row['LogP'],
            'TPSA': row['TPSA'],
            'NumRings': row['NumRings'],
            'NumDoubleBonds': row['NumDoubleBonds']
        }
    ), axis=1
)

# Rearrange columns for clarity
desired_order = [
    'SMILES', 'Corrected_SMILES', 'SMILES_Corrected', 'SMILES_Valid',
    'Chromophore', 'Auxochrome', 'MolWeight', 'LogP', 'TPSA',
    'NumRings', 'NumDoubleBonds', 'Estimated Color'
]
# Check if all desired columns are present
missing_columns = [col for col in desired_order if col not in dye_data.columns]
if missing_columns:
    for col in missing_columns:
        dye_data[col] = None  # or pd.NA

dye_data = dye_data[desired_order]

# Save the dataset to a new CSV file
output_file = 'output_dye_colors_enhanced_with_corrections.csv'
dye_data.to_csv(output_file, index=False)
print(f"Processing complete. Output saved to '{output_file}'.")

# Optionally download the result file
files.download(output_file)